# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)


## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
# I just modified the code provided originally by Udacity while retaining it to some extent (and also while retaining that 
# original code as is it is executed in the next step) in order to observe the number of unique words in sentences of the 
# scenes, number of sentences and average number of sentences,in addition to number of lines and average number of words in each
# line of the script.

view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

#print('Dataset Stats')
#print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

#lines = text.split('\n')
#print('Number of lines: {}'.format(len(lines)))
#word_count_line = [len(line.split()) for line in lines]
#print('Average number of words in each line: {}'.format(np.average(word_count_line)))

#print()
#print('The lines {} to {}:'.format(*view_line_range))
#print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))


print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of scenes: 54617
Average number of sentences in each scene: 0.0
Number of lines: 54617
Average number of words in each line: 11.088379076111833

The sentences 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you

In [3]:
#This is the Original code provided by Udacity which was executed and we can observe the results below

view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int = {word: integer for integer,word in enumerate(vocab)}
    int_to_vocab = {integer: word for integer,word in enumerate(vocab)}
    return (vocab_to_int, int_to_vocab)
    
    # return tuple
    #return (None, None)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokens_dict = {    '.': '<PERIOD>',
                       ',': '<COMMA>',
                       '"': '<QUOTATION_MARK>',
                       ';': '<SEMICOLON>',
                       '!': '<EXCLAMATION_MARK>',
                       '?': '<QUESTION_MARK>',
                       '(': '<LEFT_PAREN>',
                       ')': '<RIGHT_PAREN>',
                       '-': '<DASH>',
                       '?': '<QUESTION_MARK>',
                       '\n': '<NEW_LINE>'    }
    return tokens_dict
        
    #return None

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [8]:
max(int_to_vocab)

21387

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    print("GPU is Available!")

GPU is Available!


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [10]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    num_batches = len(words)-sequence_length-1
    feature_tensors = np.zeros((num_batches, sequence_length), dtype=int)
    target_tensors = np.zeros(num_batches, dtype=int)
    for lower in range(0, len(words)-sequence_length-1):
        feature_tensors[lower] = words[lower:lower+sequence_length]
        target_sensors[lower] = words[lower+sequence_length]
        
    data = TensorDataset(torch.from_numpy(feature_tensors), torch.from_numpy(target_tensors))
    data_loader = DataLoader(data, shuffle=True, batch_size = batch_size)
    
    return data_loader
    
    # return None

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [11]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  1,   2,   3,   4,   5],
        [  8,   9,  10,  11,  12],
        [ 22,  23,  24,  25,  26],
        [ 38,  39,  40,  41,  42],
        [ 34,  35,  36,  37,  38],
        [  7,   8,   9,  10,  11],
        [ 25,  26,  27,  28,  29],
        [ 35,  36,  37,  38,  39],
        [  9,  10,  11,  12,  13],
        [ 26,  27,  28,  29,  30]])

torch.Size([10])
tensor([  6,  13,  27,  43,  39,  12,  30,  40,  14,  31])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [12]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # define model layers
        
        # embedding and LSTM layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)
                
        # linear layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        
        # set the batch size
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.embed(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # calculate fully-connected layer output
        output = self.fc(lstm_out)
        
        # reshape fully-connected layer output to be batch size first
        output = output.view(batch_size, -1, self.output_size)
        
        # get last batch of labels 
        out = output[:, -1] 
        
        # return one batch of output word scores and the hidden state
        return out, hidden
           
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # define the hidden state weights
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [22]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    grad_clip = 5
    
    # move data to GPU, if available
    if train_on_gpu:
        rnn.cuda()

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    # Detach any passed in hidden state from its history
    hid = tuple([each.data for each in hidden])

    # zero accumulated gradients
    rnn.zero_grad()

    # move tensors to GPU, if it's available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
        
    # get the output from the model
    out, hid = rnn(inp, hid)
    
    # calculate the loss
    loss = criterion(out, target)
    
    # perform a back propagation step
    loss.backward()
    
    # clip gradients in order to prevent "exploding gradients" problem
    nn.utils.clip_grad_norm_(rnn.parameters(), grad_clip)
    
    # optimization step - update weights in the NN
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [23]:
#I later tried executing this code and observed a consistent amount of decrease in training loss by using the following parameters:
# sequence_length = 10, batch_size = 128, learning_rate = 0.01, embedding_dim = 300, hidden_dim = 400, n_layers = 2,
# num_epochs = 10, show_every_n_batches = 500.

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n' .format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

In [14]:
# I initially tried and executed this code to get an idea about how much time, how many number of steps and how many words it is 
# taking per each batch in order to complete one epoch with sequence_length = 10, batch_size = 128, learning rate = 0.01, 
# embedding_dim = 300, hidden_dim = 400, num_epochs = 10, show_n_every_batches = 500.

#And I observed that it is taking 482 secs(8 min 33 secs),taking 13 steps and taking 6969 words per batch to complete one epoch
#with the said respective parametric values.

import time
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

f_out = open('./note_output.txt', 'w')
f_out.write('start\n')

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    f_out.write('train_rnn\n')
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        start = time.time()
        epoch_start  = time.time()
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                message = 'Epoch: {:>4}/{:<4} Batch: {:>4}/{:<4}  Loss: {:.4} ; Took: {} seconds \n'.format(
                    epoch_i, n_epochs, 
                    batch_i, n_batches,
                    np.average(batch_losses), int(time.time()-start ))
                print(message)
                f_out.write(message)
                f_out.flush()
                start = time.time()
                batch_losses = []

        print('Epoch {} took {} seconds in total'.format(epoch_i, int(time.time()-epoch_start)))
        print('===========\n')
        helper.save_model('./save/trained_rnn', rnn)
        print('Model Trained and Saved')

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [24]:
print(f"# of unique words: {len(vocab_to_int)}")

# of unique words: 21388


In [25]:
# Data params
# Sequence Length
sequence_length = 10 # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [20]:
# Training parameters
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 400
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [17]:
# I initially tried with the code in (step 14)(where i did some modifications to print the Epoch and loss which was originally 
# given by Udacity) and tried to print to get an idea about how much time, how many number of steps, how many words it is taking
# per each batch and by how much amount the training loss is consistently decreasing in order to complete one epoch with the 
# following parameters:
 
# sequence_length = 10, batch_size = 128, learning rate = 0.01, embedding_dim = 300, hidden_dim = 400, num_epochs = 10,
# n_layers = 2, show_every_n_batches = 500.

# And I observed that it is taking 482 secs(8 min 33 secs),taking 13 steps and taking 6969 words per batch to complete one epoch
# with the said number of respective parametric values and the model was convering faster and learning at a faster phase 
# (though inspite of observing minute fluctations in between during training(ie.in training loss)) the model was showing a
# consistent amount of decrease in training loss.

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10   Batch:  500/6969  Loss: 5.395 ; Took: 34 seconds 

Epoch:    1/10   Batch: 1000/6969  Loss: 4.777 ; Took: 34 seconds 

Epoch:    1/10   Batch: 1500/6969  Loss: 4.588 ; Took: 34 seconds 

Epoch:    1/10   Batch: 2000/6969  Loss: 4.477 ; Took: 34 seconds 

Epoch:    1/10   Batch: 2500/6969  Loss: 4.381 ; Took: 34 seconds 

Epoch:    1/10   Batch: 3000/6969  Loss: 4.331 ; Took: 34 seconds 

Epoch:    1/10   Batch: 3500/6969  Loss: 4.272 ; Took: 34 seconds 

Epoch:    1/10   Batch: 4000/6969  Loss: 4.223 ; Took: 34 seconds 

Epoch:    1/10   Batch: 4500/6969  Loss: 4.168 ; Took: 34 seconds 

Epoch:    1/10   Batch: 5000/6969  Loss: 4.181 ; Took: 34 seconds 

Epoch:    1/10   Batch: 5500/6969  Loss: 4.161 ; Took: 34 seconds 

Epoch:    1/10   Batch: 6000/6969  Loss: 4.125 ; Took: 34 seconds 

Epoch:    1/10   Batch: 6500/6969  Loss: 4.11 ; Took: 34 seconds 

Epoch 1 took 482 seconds in total



/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved
Epoch:    2/10   Batch:  500/6969  Loss: 4.001 ; Took: 34 seconds 

Epoch:    2/10   Batch: 1000/6969  Loss: 3.9 ; Took: 34 seconds 

Epoch:    2/10   Batch: 1500/6969  Loss: 3.888 ; Took: 34 seconds 

Epoch:    2/10   Batch: 2000/6969  Loss: 3.884 ; Took: 34 seconds 

Epoch:    2/10   Batch: 2500/6969  Loss: 3.882 ; Took: 34 seconds 

Epoch:    2/10   Batch: 3000/6969  Loss: 3.876 ; Took: 34 seconds 

Epoch:    2/10   Batch: 3500/6969  Loss: 3.87 ; Took: 34 seconds 

Epoch:    2/10   Batch: 4000/6969  Loss: 3.855 ; Took: 34 seconds 

Epoch:    2/10   Batch: 4500/6969  Loss: 3.835 ; Took: 34 seconds 

Epoch:    2/10   Batch: 5000/6969  Loss: 3.866 ; Took: 34 seconds 

Epoch:    2/10   Batch: 5500/6969  Loss: 3.87 ; Took: 34 seconds 

Epoch:    2/10   Batch: 6000/6969  Loss: 3.86 ; Took: 34 seconds 

Epoch:    2/10   Batch: 6500/6969  Loss: 3.848 ; Took: 34 seconds 

Epoch 2 took 483 seconds in total

Model Trained and Saved
Epoch:    3/10   Batch:  500/6969  Los

In [26]:
# I later tried executing this code after executing the code in (step 23) (which was the original code provided by Udacity for 
# printing epoch and training loss and to save the model after it is trained with the training loss results) and observed a 
# consistent amount of decrease in training loss by using the following parameters:
# with sequence_length = 10, batch_size = 128, learning_rate = 0.01, embedding_dim = 300, hidden_dim = 400, n_layers = 2,
# num_epochs = 10, show_every_n-batches = 500.

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 10 epoch(s)...
Epoch:    1/10    Loss: 5.329535250663757

Epoch:    1/10    Loss: 4.746874620914459

Epoch:    1/10    Loss: 4.515741340637207

Epoch:    1/10    Loss: 4.388403485298157

Epoch:    1/10    Loss: 4.338966270923614

Epoch:    1/10    Loss: 4.249724737644195

Epoch:    1/10    Loss: 4.228100421428681

Epoch:    1/10    Loss: 4.158835737228394

Epoch:    1/10    Loss: 4.139732879638672

Epoch:    1/10    Loss: 4.121392249107361

Epoch:    1/10    Loss: 4.108116096496582

Epoch:    1/10    Loss: 4.091676482677459

Epoch:    1/10    Loss: 4.076794068336487

Epoch:    2/10    Loss: 3.9589022204848883

Epoch:    2/10    Loss: 3.8401034898757933

Epoch:    2/10    Loss: 3.847231439113617

Epoch:    2/10    Loss: 3.835161545276642

Epoch:    2/10    Loss: 3.8504683203697203

Epoch:    2/10    Loss: 3.817565668106079

Epoch:    2/10    Loss: 3.841586326599121

Epoch:    2/10    Loss: 3.8546676568984983

Epoch:    2/10    Loss: 3.8081846652030946

Epoch:    2/10    Los

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

Answer:(write answer,here)
*sequence_length=10
I didn't find any specific recommendations about sequence length,the common answer is, "it depends."
I noticed that shorter sequence length makes faster iterations with good improvements in the loss.

I started with sequence length = 200 and every epoch was literally taking hours.Then i tried with sequence length = 50 and each epoch took more than an hour(as it was training 13938 words/per specified number of batch) which was comparitively better than the previous sequence length value.Then i tried with sequence length = 12 and observed that the model was convering a bit faster(i.e <15 minutes per epoch,taking 6969 words/per specified number of batch) while almost not affecting the loss.So i realized Decreasing the sequence length was the most thoughtful decision to implement in this project as it allowed the model to train much faster.
so finally i have set the sequence length = 10 and observed that the model was convering at very good rate and at a faster phase,(i.e <10 minutes per epoch,taking 6969 words/batch)which also allowed me to experiment with other parameters.

*Batch size = 128
According to the course,recommended starting point is 32.32 to 256 are potentially good starting values.Larger size may have computational boost due to optimized matrix multiplication,and it can stuck in local minimum.

I tried with Batch size = 64, it was slightly faster and was producing slightly better loss.

*num_epochs = 10
I tried and achieved desired consistent amount of decrease in training loss with 10 epochs and so i retained this value.

*Learning_rate = 0.001
From the class we came to know about different learning rates.
0.01 is a good starting value:0.001,0.0001,0.1 are also good values.
I tried 0.01, but the network was not learning, while 0.001 allows the network to improve the loss faster.

*Embedding dimension = 300
The example architectures used embedding dimension from 100 for text summarization and to 1000 for translation.
After experimented with other values (e.g., 100,200),I chose a value and observed that the network was capable to learn in a better way.

Long Short-Term Memory (LSTM) or Gated Recurrent Unit (GRU)?:There is no universal advice on whether to use LSTM or GRU.
So I chose LSTM and achieved the desired results.

*Hidden dimension = 400
The example architectures used size from 200 for text summarization and to 1000 for speech recognition for large vocabulary.
I chose 400 because my embedding dimension was 300.Intuitively,the hidden dimension should be greater than the embedding dimension.
I also tried 300 while using 200 for the embedding dimension.

*Number of RNN layers = 2
General recommendation is to choose two or three layers.I chose three layers and observed not much of an improvement and 
so chosen 2 layers and got a consistent amount of decrease in training loss.

So with my finally chosen hypermeters of sequence_length = 10, batch_size = 128, learning_rate = 0.001, embedding_dim = 300, hidden_dim = 400, n_layers = 2, num_epochs = 10, show_every_n_batches = 500, I was able to achieve the training loss which was
less than 3.5.


---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [29]:
# After executing the codes in step 23 (i.e with the Udacity Original code,To print and save Training Model loss results) and 
# step 26(printing training loss at each epoch) 

# run the cell multiple times to get different results!
gen_length = 200      # modify the length to your preference
prime_word = 'elaine' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

elaine: filth of the day, i don't know.

elaine: what do you mean you don't know?

jerry: yeah. i don't know.

george: oh, my god...

kramer: oh, no, no, no. i'm not getting rid of the building with a girl in a wheelchair.

jerry: oh, no.

george: i got to get the manager back.

kramer: oh! i don't want to go.

george: oh, no, no, i don't think so.

jerry: well, i'm sure it's a good idea for the job.

jerry:(to george) you know, you gotta go to your apartment!

kramer: yeah. hey, jerry.

jerry: hi mike.

george: hey, how you doing?

elaine:(shouts) i can't believe this, it's like a grubby scrub.

elaine: oh, well, you know i just assumed you don't want to go into the shop


#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [31]:
# save script to a text file 
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.